In [2]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
#!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz --user
nlp = en_core_web_sm.load()
import pandas as pd
import json
!pip install bs4 --user
from bs4 import BeautifulSoup
import requests
import re
import os
import mmap

You are using pip version 18.0, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Load data

In [72]:
with open('data.json', 'r') as f:
    dict_json_weighted = json.load(f)
object_1 = list(dict_json_weighted.keys())[4]
for i in range(len(dict_json_weighted[object_1]['log'])):
    print(str(i)+"-"+dict_json_weighted[object_1]['log'][i]['text'])

0-Can you help me book a 1 star hotel that has free parking please?
1-I am sorry there are no 1 star hotels in any area,can I get a price range and area please?
2-Let me think about that. What are some good local restaurants?
3-There are over 100 restaurants in the city. Do you have a certain type of food in mind?
4-No, I don't. Surprise me!
5-Sure! Bedouin is african food located in the centre. Would that do?
6-Let me think about that. Can you help me book a 4 star hotel?
7-Yes I found 4 star hotel,do you want me to make reservation?
8-Do they have internet access and can you tell me their price range and postcode?
9-Yes, the University Arms Hotel has internet access. It is expensive, and its postcode is cb21ad. Would you like to book it?
10-No, that's ok. I am also looking for a restaurant that serves indian food in the cheap price range.
11-May I recommend the gandhi in the centre? It fits your criteria. 
12-That sounds good. Can you book me a table for 5 at 12 on Thursday?
13-I'm s

## Load ontology

In [111]:
with open('ontology.json', 'r') as o:
    ontology = json.load(o)
action=['inform', 'request', 'select', 'recommend', 'not found', 'request booking', 'offer booking', 
        'inform booked', 'decline booking', 'welcome','greet', 'bye', 'reqmore' ]
topic= ['hotel', 'restaurant', 'taxi', 'train', 'attraction', 'police', 'hospital']

In [113]:
top=[]
act=[]
ont=list(ontology.keys())
for i in range(len(dict_json_weighted[object_1]['log'])):
    print(dict_json_weighted[object_1]['log'][i]['text'])
    sentence =nlp (dict_json_weighted[object_1]['log'][i]['text'])
    for x in [y for y in sentence if y.pos_ != 'PUNCT']:
        if str(x).lower() in topic:
            if str(x).lower() not in top:
                top.append(str(x).lower()) 
        if x.lemma_ in action:
            if x.lemma_ not in act:
                act.append(x.lemma_)
        #for z in ont:
            #if str(x) in str(ontology[z]):
                #print(z+" :"+str(x))           
print("topics: "+str(top))
print("actions: "+str(act))


Can you help me book a 1 star hotel that has free parking please?
I am sorry there are no 1 star hotels in any area,can I get a price range and area please?
Let me think about that. What are some good local restaurants?
There are over 100 restaurants in the city. Do you have a certain type of food in mind?
No, I don't. Surprise me!
Sure! Bedouin is african food located in the centre. Would that do?
Let me think about that. Can you help me book a 4 star hotel?
Yes I found 4 star hotel,do you want me to make reservation?
Do they have internet access and can you tell me their price range and postcode?
Yes, the University Arms Hotel has internet access. It is expensive, and its postcode is cb21ad. Would you like to book it?
No, that's ok. I am also looking for a restaurant that serves indian food in the cheap price range.
May I recommend the gandhi in the centre? It fits your criteria. 
That sounds good. Can you book me a table for 5 at 12 on Thursday?
I'm sorry, they don't have tables ava

## Apply nlp for each sentence 

In [3]:
for i in range(len(dict_json_weighted[object_1]['log'])):
    print(str(i)+"-"+dict_json_weighted[object_1]['log'][i]['text'])
    sentence =nlp (dict_json_weighted[object_1]['log'][i]['text'])
    ### create a file for each sentence
    result= open("Data/"+object_1+str(i)+".txt","w+")
    ### print the label
    #print("Verbs:", [token.lemma_ for token in sentence if token.pos_ == "VERB"])
    for X in (sentence.ents) :
        print([(X.text, X.label_) ])
        r = open("Data/"+object_1+str(i)+".txt", "r")     
        if str(X.label_) not in r.read():
            print(X.label_+"**else")
            result.write("~["+X.label_+"]\n")
            result.write("   "+X.text+"\n")
        elif "~["+X.label_+"]" in r.read():
            print(X.label_+"**if")
            result.write("   "+X.text+"\n")
                
        
    #labels = [x.label_ for x in sentence.ents]
    #print(len(sentence.ents))
    ### print the labels in each sentence
    #print(Counter(labels))
    ### highlight named entities and their labels in a text
    displacy.render(sentence, jupyter=True, style='ent')
    ### display the sentence and its dependencies 
    #displacy.render(sentence, style='dep', jupyter = True, options = {'distance': 120})
    
    ###apply part-of-speech tagging and lemmatize the sentence.
    for x in [y for y in sentence if y.pos_ != 'PUNCT']:
        print ((x.orth_,x.pos_, x.lemma_))
    print ("******************************************************")
    

0-I need a train going to cambridge. 
[('cambridge', 'GPE')]
GPE**else


('I', 'PRON', '-PRON-')
('need', 'VERB', 'need')
('a', 'DET', 'a')
('train', 'NOUN', 'train')
('going', 'VERB', 'go')
('to', 'ADP', 'to')
('cambridge', 'PROPN', 'cambridge')
******************************************************
1-Absolutely! What day and time would you like to leave?
[('What day', 'DATE')]
DATE**else


('Absolutely', 'ADV', 'absolutely')
('What', 'PRON', 'what')
('day', 'NOUN', 'day')
('and', 'CCONJ', 'and')
('time', 'NOUN', 'time')
('would', 'AUX', 'would')
('you', 'PRON', '-PRON-')
('like', 'VERB', 'like')
('to', 'PART', 'to')
('leave', 'VERB', 'leave')
******************************************************
2-On Thursday any time after 21:00.
[('Thursday', 'DATE')]
DATE**else


('On', 'ADP', 'on')
('Thursday', 'PROPN', 'Thursday')
('any', 'DET', 'any')
('time', 'NOUN', 'time')
('after', 'ADP', 'after')
('21:00', 'NUM', '21:00')
******************************************************
3-Where will you be departing from? 


/usr/lib/python3.5/runpy.py:184: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


...

/usr/lib/python3.5/runpy.py:184: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


('It', 'PRON', '-PRON-')
('is', 'AUX', 'be')
('free', 'ADJ', 'free')
('to', 'PART', 'to')
('enter', 'VERB', 'enter')
('and', 'CCONJ', 'and')
('type', 'VERB', 'type')
('is', 'AUX', 'be')
('architecture', 'NOUN', 'architecture')
('May', 'AUX', 'may')
('I', 'PRON', '-PRON-')
('help', 'VERB', 'help')
('you', 'PRON', '-PRON-')
('with', 'ADP', 'with')
('anything', 'PRON', 'anything')
('else', 'ADV', 'else')
******************************************************
14-No, you have been very helpful today, thank you.  I am all set.  Goodbye.
[('today', 'DATE')]
DATE**else


('No', 'INTJ', 'no')
('you', 'PRON', '-PRON-')
('have', 'AUX', 'have')
('been', 'AUX', 'be')
('very', 'ADV', 'very')
('helpful', 'ADJ', 'helpful')
('today', 'NOUN', 'today')
('thank', 'VERB', 'thank')
('you', 'PRON', '-PRON-')
(' ', 'SPACE', ' ')
('I', 'PRON', '-PRON-')
('am', 'AUX', 'be')
('all', 'ADV', 'all')
('set', 'VERB', 'set')
(' ', 'SPACE', ' ')
('Goodbye', 'PROPN', 'Goodbye')
******************************************************
15-Thank you for using our service!


/usr/lib/python3.5/runpy.py:184: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


('Thank', 'VERB', 'thank')
('you', 'PRON', '-PRON-')
('for', 'ADP', 'for')
('using', 'VERB', 'use')
('our', 'PRON', '-PRON-')
('service', 'NOUN', 'service')
******************************************************
